# Regression Problem

### 1. Import necessary libraries:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from mltools.regression_tools import LinearRegressor

### 2. Load data as a Pandas DataFrame:

In [2]:
path_irrad = "../data/G07A_DATOS_IRRAD.csv"
df_orig_irrad = pd.read_csv(path_irrad)

# Parse the date (string) for it to be a datetime with format yyyy/mm/dd
df_orig_irrad['FECHA'] = pd.to_datetime(df_orig_irrad['FECHA'], format='%Y-%m-%d')

s = df_orig_irrad.loc[:,'FECHA']
df_orig_irrad['FECHA'] =  s.dt.date

In [3]:
path_util = "../data/G07A_DATOS_UTIL.csv"
df_orig_util = pd.read_csv(path_util)

# Parse the date (string) for it to be a datetime with format yyyy/mm/dd
df_orig_util['FECHA'] = pd.to_datetime(df_orig_util['FECHA'], format='%Y-%m-%d')

s = df_orig_util.loc[:,'FECHA']
df_orig_util['FECHA'] =  s.dt.date

In [4]:
df = pd.merge(df_orig_irrad, df_orig_util, on=['FECHA', 'ANNO', 'MES', 'DIA', 'DIASEM'] , how='inner')
df.head()

,FECHA,IRRADH00,IRRADH03,IRRADH06,IRRADH09,IRRADH12,IRRADH15,IRRADH18,IRRADH21,ANNO,...,DIA,DIASEM,UTILH00,UTILH03,UTILH06,UTILH09,UTILH12,UTILH15,UTILH18,UTILH21
0,2015-01-01,0.0,0.0,0.0,414536.22,1193085.20,481816.38,0.0,0.0,2015,...,1,3,0.0,0.0,0.027848,0.235443,0.263291,0.039241,0.0,0.0
1,2015-01-02,0.0,0.0,0.0,319990.20,926238.10,182854.10,0.0,0.0,2015,...,2,4,0.0,0.0,0.027848,0.181857,0.205063,0.024051,0.0,0.0
2,2015-01-03,0.0,0.0,0.0,403464.62,1146347.20,360073.94,0.0,0.0,2015,...,3,5,0.0,0.0,0.029536,0.237975,0.235865,0.036287,0.0,0.0
3,2015-01-04,0.0,0.0,0.0,349597.03,1016177.06,315783.62,0.0,0.0,2015,...,4,6,0.0,0.0,0.029114,0.196203,0.210970,0.026160,0.0,0.0
4,2015-01-05,0.0,0.0,0.0,408705.38,1090662.90,532075.10,0.0,0.0,2015,...,5,0,0.0,0.0,0.028270,0.205485,0.228270,0.036287,0.0,0.0


### 3. Split the data into training and test sets:

In [8]:
# Define input and output matrices:
INPUTS = ['IRRADH00', 'IRRADH03', 'IRRADH06', 'IRRADH09', 'IRRADH12',
          'IRRADH15', 'IRRADH18', 'IRRADH21', 'ANNO', 'MES', 'DIA', 'DIASEM']
OUTPUTS = ['UTILH00', 'UTILH03', 'UTILH06', 'UTILH09', 
          'UTILH12', 'UTILH15', 'UTILH18', 'UTILH21']
X = df[INPUTS]
Y = df[OUTPUTS]

# Split:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.3,  # Percentage of test data
                                                    random_state=0) # Seed for replication

In [9]:
# Create dataset to store model predictions:
dfTR_eval = X_train.copy()
dfTR_eval[OUTPUTS] = Y_train

dfTS_eval = X_test.copy()
dfTS_eval[OUTPUTS] = Y_test

### 4. Training of Linear Regression Model:

In [10]:
# Inputs of the model
INPUTS_LR = INPUTS

# Preparation of the numeric variables by scaling
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Creation of a preprocessor to perform the steps defined above
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, INPUTS)])

# Establishment the Linear Regression Model within a Pipeline
pipe = Pipeline(steps=[('Prep',preprocessor), # Preprocess the variables when training the model 
                        ('LinReg',LinearRegressor(fit_intercept=True))]) 

# Use of Grid Search Cross Validation to find the best hyperparameter for the model in the grid defined 
nFolds = 10
param = {} # In regression there is no hyperparameter
LR_fit = GridSearchCV(estimator=pipe, # Structure of the model to use
                        param_grid=param, # Defined grid to search in
                        n_jobs=-1, # Number of cores to use (parallelize)
                        scoring = 'neg_root_mean_squared_error', # Scoring function
                        cv=nFolds) # Number of Folds 
LR_fit.fit(X_train[INPUTS], Y_train) # Search in grid

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('Prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['IRRADH00',
                                                                          'IRRADH03',
                                                                          'IRRADH06',
                                                                          'IRRADH09',
                                                                          'IRRADH12',
                                                                          'IRRADH15',
                                                                          'IRRADH18',
                                                                          'IRRADH21',
                                                                          'ANNO',
                                                                          'MES',
                                                                          'DIA',
                                                                          'DIASEM'])])),
                                       ('LinReg', LinearRegressor())]),
             n_jobs=-1, param_grid={}, scoring='neg_root_mean_squared_error')